GAN-BERT (in Pytorch and compatible with HuggingFace) : https://github.com/crux82/ganbert-pytorch

In [ ]:
# !pip install -U transformers

In [ ]:
import os
import re
import urllib.request
import random
import time
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, BertForSequenceClassification, GPT2LMHeadModel, set_seed

# Set random seed for reproducibility
manualSeed = 0
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)
set_seed(manualSeed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(manualSeed)

## Inputs




In [ ]:
# Number of training epochs
num_epochs = 10

# Learning rate for optimizers
lr = 5e-5

# number of classes in the dataset
n_class_dataset = 6

# name of each class in the dataset
label2class = {0: "human", 1: "chatGPT", 2: "cohere", 3: "davinci", 4: "bloomz", 5: "dolly"}

# batch size
batch_size = 64

# max sequence length for BERT models
max_length_bert = 64 # Similar to GAN-BERT

# generator model
generator_type = 'G1'
assert generator_type in ['G1', 'G2', 'G3']

# Input dimension of the G1 network (the latent vector z)
d_in = 100

# Output dimension of the G1 network
d_out = 768

# Dropout parameter
p_dropout = 0.2

# beta1 and beta2 for the ADAM optimizers
betas_ADAM = (0.9, 0.999) # Note: no values reported in the paper

# Number of GPUs available
ngpu = 1 # use two GPU for kaggle

# Decide which device we want to run on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# lambda values in the loss functions
lambda_score = 1
lambda_feature_matching = 1

In [ ]:
if generator_type in ['G2', 'G3']:
    raise NotImplementedError # For G3: we have to fine-tune distilGP2 model over a few labeled samples

In [ ]:
length_dataset_train = 71027
length_dataset_val = 3000
labeled_fraction_train = 0.01 # select this parameter in range of [0, 1]

# Split the training dataset into labeled and unlabeled samples
random_state_perm = np.random.RandomState(manualSeed)
n_labeled_samples = int(length_dataset_train*labeled_fraction_train)
print('labeled fraction: {:.2f} %'.format(labeled_fraction_train*100))
labeled_indices_train = random_state_perm.permutation(length_dataset_train)[:n_labeled_samples]
labeled_indices_val = np.arange(length_dataset_val)

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance_train = True
if apply_balance_train:
    balance_factor_train = int(1/labeled_fraction_train)
    balance_factor_train = 3*int(np.log2(balance_factor_train)) # with an additional factor of 3
    if balance_factor_train < 1:
        balance_factor_train = 1
    print('balance factor train: {0}'.format(balance_factor_train))
else:
    balance_factor_train = None

## Data




https://github.com/mbzuai-nlp/SemEval2024-task8

In [ ]:
url_train = "https://dl.dropboxusercontent.com/scl/fi/ux5ke4t5id230976pm0xt/subtaskB_train.jsonl?rlkey=l9tgwwhw75sbap08rap67r8vh&dl=0"
urllib.request.urlretrieve(url_train, "subtaskB_train.jsonl")

In [ ]:
url_dev = "https://dl.dropboxusercontent.com/scl/fi/sb4kzf4ifxzgn4ottz3am/subtaskB_dev.jsonl?rlkey=qqxkcjqh09ecjdji192h95opt&dl=0"
urllib.request.urlretrieve(url_dev, "subtaskB_dev.jsonl")

In [ ]:
class SubtaskBDataset(Dataset):
    def __init__(self, file_path, labeled_indices, apply_balance=False, balance_factor=None):
        self.data = []
        self.models = []
        self.sources = []
        with open(file_path, 'r') as file:
            for i, line in enumerate(file, 0):
                sample = json.loads(line)
                model = sample["model"]
                source = sample["source"]
                text = sample["text"]
                y_label = sample["label"]
                input_prompt = "Model: {0}".format(model)
                is_labeled = (i in labeled_indices)
                item = (text, y_label, input_prompt, is_labeled)
                if apply_balance:
                    if is_labeled:
                        for _ in range(balance_factor):
                            self.data.append(item)
                    else:
                        self.data.append(item)
                else:
                    self.data.append(item)
                self.models.append(model)
                self.sources.append(source)
        self.models = np.unique(self.models)
        self.sources = np.unique(self.sources)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
          return self.data[idx]

In [ ]:
datasetTrain = SubtaskBDataset("/content/subtaskB_train.jsonl", labeled_indices=labeled_indices_train, apply_balance=apply_balance_train, balance_factor=balance_factor_train)
dataloaderTrain =  DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

print(datasetTrain.__len__())
print(datasetTrain.models)
print(datasetTrain.sources)
print(datasetTrain.__getitem__(0))

In [ ]:
datasetVal = SubtaskBDataset("/content/subtaskB_dev.jsonl", labeled_indices=labeled_indices_val, apply_balance=False, balance_factor=None)
dataloaderVal =  DataLoader(datasetVal, batch_size=batch_size, shuffle=False)

print(datasetVal.__len__())
print(datasetVal.models)
print(datasetVal.sources)
print(datasetVal.__getitem__(0))

# The fine-tuned models/tokenizers

In [ ]:
def loadGPT2(model_path="alinourian/DistilGPT2-SemEval2024"):
    # Load the fine-tuned model and tokenizer
    modelGPT2 = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizerGPT2 = AutoTokenizer.from_pretrained(model_path)
    return tokenizerGPT2, modelGPT2

In [ ]:
def loadBERT(model_path="bert-base-cased"):
    # We do not use the fine-tuned BERT model of "mohammadhossein/SemEvalTask8_SubTaskB"
    # Load the model and tokenizer
    modelBERT = BertForSequenceClassification.from_pretrained(model_path)
    tokenizerBERT = AutoTokenizer.from_pretrained(model_path)
    return tokenizerBERT, modelBERT

In [ ]:
def blockGPT2(input_prompt, tokenizerGPT2, modelGPT2):
    # Example of an input text : ["Model: chatGPT"]
    encoding = tokenizerGPT2(input_prompt, padding=True, return_tensors="pt").to(device)
    # https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id
    output = modelGPT2.generate(
    **encoding,
    max_length=400,
    num_beams=1,
    temperature=0.8,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizerGPT2.eos_token_id
    )
    # A batch of generated texts
    generated_text = tokenizerGPT2.batch_decode(output, skip_special_tokens=True)
    # Remove the model names from the generated texts
    generated_text = [re.sub(r'Model:.+Text: ','', text, flags=re.IGNORECASE) for text in generated_text]

    return generated_text

In [ ]:
def blockBERT(input_text, tokenizerBERT, modelBERT):
    # https://discuss.huggingface.co/t/how-to-get-cls-embeddings-from-bertfortokenclassification-model/9276/2
    inputs = tokenizerBERT(input_text, return_tensors="pt", truncation=True, padding=True, max_length=max_length_bert).to(device)
    outputs = modelBERT(**inputs, output_hidden_states=True)
    last_hidden_states = outputs.hidden_states[-1]
    CLS_hidden_states = last_hidden_states[:, 0, :] # (bs, 768)
    return CLS_hidden_states

## Generator




In [ ]:
class GeneratorG1(nn.Module):
    def __init__(self):
        super(GeneratorG1, self).__init__()
        self.main = nn.Sequential(
            # input: Z
            nn.Linear(d_in, d_out),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=p_dropout),
            nn.Linear(d_out, d_out)
            # output: v_G
        )

    def forward(self, latent_vector):
        v_G = self.main(latent_vector)
        return v_G

In [ ]:
# Create the generator
if generator_type == 'G1':
    # Generator G1, see figure 2a
    netG = GeneratorG1()
    # Handle multi-GPU if desired
    if ngpu > 1:
        netG = nn.DataParallel(netG, device_ids=list(range(ngpu))).to(device)
    else:
        netG = netG.to(device)
    # Print the model
    print(netG)
elif generator_type == 'G3':
    # Generator G3, see figure 2c
    tokenizerGPT2, modelGPT2 = loadGPT2()
    # freeze the parameters of GPT2
    for param in modelGPT2.parameters():
        param.requires_grad = False
    tokenizerBERT_netG, modelBERT_netG = loadBERT()
    # Handle multi-GPU if desired
    if ngpu > 1:
        modelGPT2 = nn.DataParallel(modelGPT2, device_ids=list(range(ngpu))).to(device)
        modelBERT_netG = nn.DataParallel(modelBERT_netG, device_ids=list(range(ngpu))).to(device)
    else:
        modelGPT2 = modelGPT2.to(device)
        modelBERT_netG = modelBERT_netG.to(device)
    # Print the models
    print(modelGPT2)
    print(modelBERT_netG)
else:
    raise NotImplementedError

## Discriminator




In [ ]:
# Discriminator D, see figure 2d
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.seq1 = nn.Sequential(
            # input: v_G or v_B
            nn.Dropout(p=p_dropout),
            nn.Linear(d_out, d_out))
        self.seq2 = nn.Sequential(
            nn.LeakyReLU(0.2),
            nn.Dropout(p=p_dropout),
            nn.Linear(d_out, 1 + n_class_dataset), # +1 for the probability of this sample being fake/real.
            # output: logits, format: [fake score, dataset classes]
        )
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input):
        features = self.seq1(input) # required for the feature matching loss
        logits = self.seq2(features)
        probs = self.softmax(logits)
        return features, logits, probs

In [ ]:
# Create the Discriminator
netD = Discriminator()

# Handle multi-GPU if desired
if ngpu > 1:
    netD = nn.DataParallel(netD, device_ids=list(range(ngpu))).to(device)
else:
    netD = netD.to(device)

# Print the model
print(netD)

## BERT (red block)

In [ ]:
# BERT block for processing the real dataset, pictured in red in figure 1
tokenizerBERT_red, modelBERT_red = loadBERT()

# We don't freeze the parameters of modelBERT_red anymore

# Handle multi-GPU if desired
if ngpu > 1:
    modelBERT_red = nn.DataParallel(modelBERT_red, device_ids=list(range(ngpu))).to(device)
else:
    modelBERT_red = modelBERT_red.to(device)

## Accuracy

In [ ]:
def get_accuracy(dataloader):
    modelBERT_red.eval()
    netD.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloader, 0):
            text = data[0] # text samples
            y_label = data[1].to(device) # true class labels
            # output of the BERT module for real samples (CLS hidden state)
            v_B = blockBERT(text, tokenizerBERT_red, modelBERT_red)
            _, logits, _ = netD(v_B)
            _, predicted = torch.max(logits[:, 1:], 1)
            total += y_label.size(0)
            correct += (predicted == y_label).sum().item()
    accuracy = 100.0*correct/total
    modelBERT_red.train()
    netD.train()
    return accuracy

## Training




In [ ]:
criterionGAN = nn.BCEWithLogitsLoss()
criterionScore = nn.CrossEntropyLoss()

# Setup Adam optimizers for both G and D
D_vars = list(netD.parameters()) + list(modelBERT_red.parameters())

if generator_type == 'G1':
    G_vars = list(netG.parameters())
elif generator_type == 'G3':
    G_vars = list(modelBERT_netG.parameters())
else:
    raise NotImplementedError

optimizerD = optim.Adam(D_vars, lr=lr, betas=betas_ADAM)
optimizerG = optim.Adam(G_vars, lr=lr, betas=betas_ADAM)

# Establish convention for real and fake labels during training
real_label = 0.
fake_label = 1.

In [ ]:
# Training Loop

iters = 0
time_start = time.time()
print("Starting Loop ...")
# For each epoch
for epoch in range(num_epochs):
    # Validation Accuracy
    print('[%3d/%3d]\tValidation Accuracy: %.2f' %(epoch, num_epochs, get_accuracy(dataloaderVal)))

    # Training: For each batch in the dataloader
    for i, data in enumerate(dataloaderTrain, 0):

        # data format: (text, y_label, input_prompt, is_labeled)
        text = data[0] # text samples
        y_label = data[1].to(device) # true class labels
        input_prompt = data[2] # used for GPT2
        is_labeled = data[3].to(device) # identifying labeled or unlabeled training data
        b_size = y_label.size(0)

        # output of the BERT module for real samples (CLS hidden state)
        v_B = blockBERT(text, tokenizerBERT_red, modelBERT_red)

        if generator_type == 'G1':
            # Generate batch of latent vectors
            latent_vector = torch.randn(b_size, d_in, device=device)
            v_G = netG(latent_vector)
        elif generator_type == 'G3':
            generated_text = blockGPT2(input_prompt, tokenizerGPT2, modelGPT2)
            v_G = blockBERT(generated_text, tokenizerBERT_netG, modelBERT_netG)
        else:
            raise NotImplementedError

        features_real, logits_real, _ = netD(v_B)
        features_fake, logits_fake, _ = netD(v_G)

        ############################
        # Loss evaluation
        ############################
        labelGAN_real = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        labelGAN_fake = torch.full((b_size,), fake_label, dtype=torch.float, device=device)

        # It may be the case that a batch does not contain labeled examples,
        # so the "supervised loss" in this case is not evaluated
        if torch.sum(is_labeled) == 0:
            loss_D_score = 0
        else:
            loss_D_score = criterionScore(logits_real[:, 1:][is_labeled], y_label[is_labeled])

        loss_D_real = criterionGAN(logits_real[:, 0], labelGAN_real)
        loss_D_fake = criterionGAN(logits_fake[:, 0], labelGAN_fake)

        loss_D_total = loss_D_real + loss_D_fake + lambda_score*loss_D_score

        loss_G_caught = criterionGAN(logits_fake[:, 0], labelGAN_real) # fake labels are real for generator cost
        loss_G_feature_matching = torch.mean(torch.square(torch.mean(features_real, dim=0) - torch.mean(features_fake, dim=0)))

        loss_G_total = loss_G_caught + lambda_feature_matching*loss_G_feature_matching

        ############################
        # Optimization
        ############################
        # Avoid gradient accumulation
        optimizerG.zero_grad()
        optimizerD.zero_grad()

        # retain_graph = True is required since the underlying graph will be deleted after backward
        loss_G_total.backward(retain_graph=True)
        loss_D_total.backward()

        optimizerG.step()
        optimizerD.step()

        ############################
        # Training statistics
        ############################
        if i % 10 == 0:
            elapsed_time = time.time() - time_start
            print('[%3d/%3d][%3d/%3d]\tLoss_D_total: %.4f\tLoss_G_total: %.4f\tloss_D_score: %.4f\tElapsedTime: %.1f s'
                  % (epoch, num_epochs, i, len(dataloaderTrain), loss_D_total, loss_G_total, loss_D_score, elapsed_time))

        iters += 1
print("--------------------------------------------------------------------------------------------")

Accuracy

In [ ]:
get_accuracy(dataloaderVal)

In [ ]:
# get_accuracy(dataloaderTrain)